In [38]:

#RandomForestClassifier
import math
import matplotlib as mpl
import warnings
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


excelFile = r'files/sw_train_data_sample_33214.xlsx'
sample_df = pd.DataFrame(pd.read_excel(excelFile))
print(f"{sample_df.shape}")
sample_df.head()

(2051, 11)


,code,name,kPattern,k,d,dif,dea,macd,open,short,long
0,801012,农产品加工II,33214,54.193490,53.001464,4.653048,28.310486,-23.657439,-0.019959,4.203308,1.942705
1,801012,农产品加工II,33214,42.115690,53.469651,-8.987499,-17.455907,8.468408,0.199770,0.691922,0.101039
2,801012,农产品加工II,33214,13.509500,19.485156,-25.938599,-20.195254,-5.743346,0.360893,1.392745,0.721243
3,801012,农产品加工II,33214,64.128683,58.424104,-42.129105,-32.290747,-9.838358,0.432093,1.421845,0.847409
4,801012,农产品加工II,33214,85.045484,86.250993,-29.805827,-64.336337,34.530510,-0.170309,0.907151,0.225449


In [39]:
def set_0_or_1(x):
    if x > 0:
        return 1
    return 0

def toInter(x):
    return int(x + 10.5)

def percent_to_one(x):
    return int(x * 100) /100.0

def to_one_min_max_120(x):
    v = x / 120
    if  v > 1:
        v = 1
    if  v< -1:
        v =-1
    return int(v*100)/100.0

d = sample_df
d['d'] = d.k - d.d
d['d'] = (d.d /100).apply(percent_to_one)  #归一化
d['k'] = (d.k /100).apply(percent_to_one)  #归一化
d['open'] = d.open.apply(percent_to_one)   #归一化
d['macd'] = d.macd.apply(to_one_min_max_120)   #归一化
#d['short'] = d.short.apply(toInter)  ##涨幅编码，如：14 ，表示涨幅为4
#d['short'] = d.short.apply(set_0_or_1)
d['long'] = d.long .apply(toInter)
sample_df = sample_df.drop(columns=['code','kPattern', 'name','dif','dea'])
sample_df.rename(columns={'d':'kd_diff'},inplace=True)
sample_df.head()

,k,kd_diff,macd,open,short,long
0,0.54,0.01,-0.19,-0.01,1,12
1,0.42,-0.11,0.07,0.19,1,10
2,0.13,-0.05,-0.04,0.36,1,11
3,0.64,0.05,-0.08,0.43,1,11
4,0.85,-0.01,0.28,-0.17,1,10


In [40]:
data  = sample_df.values
x,y=np.split(data,indices_or_sections=(4,),axis=1) #x为数据，y为标签
y = y[:,0:1].astype('int')  #取第一列
y


array([[1],
       [1],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [41]:
import sklearn
from sklearn import model_selection
x_train,x_test,y_train,y_test =model_selection.train_test_split(x,y, train_size=0.7,test_size=0.3)

y_train = y_train.ravel()
y_test = y_test.ravel()
x_train.shape

(1435, 4)

In [42]:
n_features = 20
#核心代码
#传统决策树、随机森林算法、极端随机树关于区别:https://blog.csdn.net/hanss2/article/details/53525503
#关于其中参数的说明请看http://www.jb51.net/article/131172.htm
clf1 = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
clf2 = RandomForestClassifier(n_estimators=10,max_depth=None,min_samples_split=2, bootstrap=True)
clf3 = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, bootstrap=False)


In [30]:
clf2.fit(x_train,y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [37]:
sampel_1 = x_test[0,]
restul = clf2.predict([sampel_1])
restul

array([10])

In [43]:

#交叉验证
scores1 = cross_val_score(clf1, x_train, y_train)
scores2 = cross_val_score(clf2, x_train, y_train)
scores3 = cross_val_score(clf3, x_train, y_train)
print()
print('DecisionTreeClassifier交叉验证准确率为:'+str(scores1.mean()))    
print('RandomForestClassifier交叉验证准确率为:'+str(scores2.mean()))    
print('ExtraTreesClassifier交叉验证准确率为:'+str(scores3.mean()))


DecisionTreeClassifier交叉验证准确率为:0.6195121951219512
RandomForestClassifier交叉验证准确率为:0.6599303135888501
ExtraTreesClassifier交叉验证准确率为:0.659233449477352
